<a href="https://colab.research.google.com/github/jda-21/AI4ENG/blob/main/04%20-%20preprocesado%20limpieza%20cualitativa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Limpieza y normalización - Train Dataset

Se realiza el proceso de limpieza y estandarización del archivo **train.csv**, garantizando que todas las columnas y valores estén en un formato adecuado para el modelo. Para garantizar los datos homogéneos, eliminar valores faltantes, sin inconsistencias en texto y con el target correctamente normalizado.

Transformaciones aplicadas:

- **Estandarización de nombres de columnas:** se convierten a mayúsculas, sin tildes o caracteres especiales.

- **Normalización de la variable objetivo `RENDIMIENTO_GLOBAL`:** se unifican todas las variantes textuales *multiclass* y se convierte a una codificación numérica uniforme usada para el modelo.

- **Imputación de valores faltantes:**  
  - Numéricas → se completan con la **mediana**.  
  - Categóricas → se completan usando **distribución global** de cada columna.

Finalmente se tiene un dataframe limpio, consistente y preparado para el entrenamiento del modelo CatBoost, evitando errores en el proceso de modelado.

In [1]:
!pip install Unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 5.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from unidecode import unidecode

RAW_PATH = "/content/train.csv"
df = pd.read_csv(RAW_PATH, engine="python", on_bad_lines="skip")
print("Shape original:", df.shape)

def clean_colname(col):
    col = unidecode(str(col))
    col = col.upper().strip()
    col = col.replace(" ", "_").replace("-", "_")
    return col

df.columns = [clean_colname(c) for c in df.columns]
print("\nColumnas limpias (ejemplo):")
print(df.columns[:20])

def clean_cell_value(val):
    if pd.isna(val):
        return val
    val = unidecode(str(val))
    val = val.upper().strip()
    val = val.replace("-", " ")
    val = " ".join(val.split())
    return val

text_cols = df.select_dtypes(include=["object"]).columns.tolist()
print(f"\nColumnas de texto detectadas ({len(text_cols)}):")
print(text_cols[:20])

for c in text_cols:
    df[c] = df[c].apply(clean_cell_value)

TARGET_COL_RAW = "RENDIMIENTO_GLOBAL"
TARGET_COL_NUM = "RENDIMIENTO_GLOBAL_NUM"

if TARGET_COL_RAW in df.columns:
    print("\nValores originales de RENDIMIENTO_GLOBAL:")
    print(df[TARGET_COL_RAW].value_counts(dropna=False))

    def normalizar_rend(val):
        if pd.isna(val):
            return np.nan
        s = unidecode(str(val)).upper().strip()
        s = " ".join(s.replace("_", " ").replace("-", " ").split())
        if "MEDIO" in s and "BAJO" in s:
            return "MEDIO_BAJO"
        if "MEDIO" in s and "ALTO" in s:
            return "MEDIO_ALTO"
        if "BAJO" in s and "MEDIO" not in s and "ALTO" not in s:
            return "BAJO"
        if "ALTO" in s and "MEDIO" not in s and "BAJO" not in s:
            return "ALTO"
        return np.nan

    df["RENDIMIENTO_NORMALIZADO"] = df[TARGET_COL_RAW].apply(normalizar_rend)
    print("\nValores normalizados:")
    print(df["RENDIMIENTO_NORMALIZADO"].value_counts(dropna=False))

    map_rend = {
        "BAJO": 1,
        "MEDIO_BAJO": 2,
        "MEDIO_ALTO": 3,
        "ALTO": 4
    }

    df[TARGET_COL_NUM] = df["RENDIMIENTO_NORMALIZADO"].map(map_rend)
    before = len(df)
    df = df.dropna(subset=[TARGET_COL_NUM]).copy()
    df[TARGET_COL_NUM] = df[TARGET_COL_NUM].astype(int)
    after = len(df)

    print(f"\nFilas eliminadas por etiqueta inválida: {before-after} (de {before})")
    print("\nConteo final de clases:")
    print(df[TARGET_COL_NUM].value_counts().sort_index())
else:
    print("\n'RENDIMIENTO_GLOBAL' no existe en este dataset.")
    if "RENDIMIENTO_NORMALIZADO" not in df.columns:
        df["RENDIMIENTO_NORMALIZADO"] = np.nan
    if TARGET_COL_NUM not in df.columns:
        df[TARGET_COL_NUM] = np.nan

def clean_text_col(series):
    return (
        series.astype(str)
        .map(lambda x: unidecode(x))
        .str.upper()
        .str.strip()
    )

for c in ["E_PRGM_ACADEMICO_FE", "E_PRGM_DEPARTAMENTO"]:
    if c in df.columns:
        df[c] = clean_text_col(df[c])

num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
num_cols_no_target = [c for c in num_cols if c != TARGET_COL_NUM]

cat_cols_all = [c for c in df.columns if c not in num_cols]
cat_cols_fill = [c for c in cat_cols_all if c != "RENDIMIENTO_NORMALIZADO"]

df[num_cols_no_target] = df[num_cols_no_target].fillna(df[num_cols_no_target].median())
df[cat_cols_fill] = df[cat_cols_fill].fillna("DESCONOCIDO")

print("\nNaN totales tras limpieza:", df.isna().sum().sum())

def impute_binary_global(df_in, cat_cols, random_state=42):
    rng = np.random.default_rng(random_state)
    df_out = df_in.copy()
    for col in cat_cols:
        if col not in df_out.columns:
            continue
        mask = df_out[col] == "DESCONOCIDO"
        n_missing = mask.sum()
        if n_missing == 0:
            continue
        print(f"\n>>> Imputando columna: {col} ({n_missing} valores 'DESCONOCIDO')")
        valid = df_out.loc[~mask, col]
        probs = valid.value_counts(normalize=True)
        if probs.empty:
            values = np.array(["SI", "NO"])
            p = np.array([0.5, 0.5])
        else:
            values = probs.index.to_numpy()
            p = probs.to_numpy()
            p = p / p.sum()
        sampled = rng.choice(values, size=n_missing, p=p)
        df_out.loc[mask, col] = sampled
        if "DESCONOCIDO" in df_out[col].unique():
            print("Reemplazando 'DESCONOCIDO' por el valor más frecuente…")
            mode_val = df_out.loc[df_out[col] != "DESCONOCIDO", col].value_counts().idxmax()
            df_out.loc[df_out[col] == "DESCONOCIDO", col] = mode_val
        print(df_out[col].value_counts(dropna=False))
    return df_out

cat_cols_binarias = [
    "F_TIENELAVADORA",
    "F_TIENEAUTOMOVIL",
    "F_PRIVADO_LIBERTAD",
    "E_PAGOMATRICULAPROPIO",
    "F_TIENEINTERNET",
    "F_TIENEINTERNET.1",
    "F_TIENECOMPUTADOR",
]

df_clean = impute_binary_global(
    df_in=df,
    cat_cols=cat_cols_binarias,
    random_state=42
)

print("\nLimpieza completa. Shape final:", df_clean.shape)

Shape original: (692500, 21)

Columnas limpias (ejemplo):
Index(['ID', 'PERIODO_ACADEMICO', 'E_PRGM_ACADEMICO', 'E_PRGM_DEPARTAMENTO',
       'E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA',
       'F_ESTRATOVIVIENDA', 'F_TIENEINTERNET', 'F_EDUCACIONPADRE',
       'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL', 'E_PRIVADO_LIBERTAD',
       'E_PAGOMATRICULAPROPIO', 'F_TIENECOMPUTADOR', 'F_TIENEINTERNET.1',
       'F_EDUCACIONMADRE', 'RENDIMIENTO_GLOBAL', 'INDICADOR_1', 'INDICADOR_2',
       'INDICADOR_3'],
      dtype='object')

Columnas de texto detectadas (15):
['E_PRGM_ACADEMICO', 'E_PRGM_DEPARTAMENTO', 'E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA', 'F_ESTRATOVIVIENDA', 'F_TIENEINTERNET', 'F_EDUCACIONPADRE', 'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL', 'E_PRIVADO_LIBERTAD', 'E_PAGOMATRICULAPROPIO', 'F_TIENECOMPUTADOR', 'F_TIENEINTERNET.1', 'F_EDUCACIONMADRE', 'RENDIMIENTO_GLOBAL']

Valores originales de RENDIMIENTO_GLOBAL:
RENDIMIENTO_GLOBAL
ALTO          175619
BAJO          172987
M

In [3]:
def resumen_valores_unicos(df, max_mostrar=20):

    for col in df.columns:
        uniques = df[col].unique()
        n_unique = len(uniques)

        print(f"\n--- {col} ---")
        print(f"Unique: {n_unique}")

        if n_unique <= max_mostrar:
            print("Total:", uniques)
        else:
            # Mostrar solo los primeros valores si hay demasiados
            print("Values", uniques[:max_mostrar], "...")

resumen_valores_unicos(df_clean)


--- ID ---
Unique: 692500
Values [904256 645256 308367 470353 989032 659872  47159  11829 257869 465511
 273010 738026 858669 252472 636397 961327  52868 937873 797253  65292] ...

--- PERIODO_ACADEMICO ---
Unique: 9
Total: [20212 20203 20195 20183 20194 20213 20184 20202 20196]

--- E_PRGM_ACADEMICO ---
Unique: 755
Values ['ENFERMERIA' 'DERECHO' 'MERCADEO Y PUBLICIDAD'
 'ADMINISTRACION DE EMPRESAS' 'PSICOLOGIA' 'MEDICINA VETERINARIA'
 'INGENIERIA MECANICA' 'ADMINISTRACION EN SALUD OCUPACIONAL'
 'INGENIERIA INDUSTRIAL' 'ADMINISTRACION FINANCIERA' 'HOTELERIA Y TURISMO'
 'LICENCIATURA EN CIENCIAS SOCIALES' 'LICENCIATURA EN PEDAGOGIA INFANTIL'
 'COMUNICACION SOCIAL' 'CIENCIA POLITICA'
 'PROFESIONAL EN GESTION DE LA SEGURIDAD Y LA SALUD LABORAL'
 'MAESTRO EN MUSICA' 'INGENIERIA MECATRONICA' 'TRABAJO SOCIAL'
 'LICENCIATURA EN BIOLOGIA Y EDUCACION AMBIENTAL'] ...

--- E_PRGM_DEPARTAMENTO ---
Unique: 31
Values ['BOGOTA' 'ATLANTICO' 'SANTANDER' 'ANTIOQUIA' 'HUILA' 'SUCRE' 'CAQUETA'
 'CUNDINAM

In [4]:
df_clean.columns.tolist()

['ID',
 'PERIODO_ACADEMICO',
 'E_PRGM_ACADEMICO',
 'E_PRGM_DEPARTAMENTO',
 'E_VALORMATRICULAUNIVERSIDAD',
 'E_HORASSEMANATRABAJA',
 'F_ESTRATOVIVIENDA',
 'F_TIENEINTERNET',
 'F_EDUCACIONPADRE',
 'F_TIENELAVADORA',
 'F_TIENEAUTOMOVIL',
 'E_PRIVADO_LIBERTAD',
 'E_PAGOMATRICULAPROPIO',
 'F_TIENECOMPUTADOR',
 'F_TIENEINTERNET.1',
 'F_EDUCACIONMADRE',
 'RENDIMIENTO_GLOBAL',
 'INDICADOR_1',
 'INDICADOR_2',
 'INDICADOR_3',
 'INDICADOR_4',
 'RENDIMIENTO_NORMALIZADO',
 'RENDIMIENTO_GLOBAL_NUM']

In [ ]:
output_path = "df_TestAdj2.csv"
df1.to_csv(output_path, index=False)
print(f"Archivo guardado como: {output_path}")

Archivo guardado como: df_binaryAdj2.csv
